<a href="https://colab.research.google.com/github/adhiutama17/MKBigData/blob/main/FixTugas_TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Fix untuk Tugas MK Big Data - Text Summarization

## Daftar Isi



*   Pendahuluan
*   Teori
*   Metode
*   Uji Coba
*   Validasi
*   Test



##Pendahuluan
Era Internet menyediakan beragam informasi yang bisa diakses kapan pun dan dimana pun, misalnya berita elektronik, iklan, kursus online, hingga artikel-artikel ilmiah. Contoh terakhir menjadi salah satu sumber informasi yang wajib di dunia pendidikan khususnya pendidikan tinggi. Artikel ilmiah didefinisikan sebagai karya ilmu pengetahuan yang menyajikan fakta umum dan ditulis dengan metodologi penulisan yang baik dan benar. Konten dari artikel ilmiah dapat berupa penelitian, ulasan teori, hingga ide baru seseorang didasarkan suatu permasalahan. Menurut World Bank , dari tahun 2000 hingga 2018 tercatat lebih dari 2,5 juta artikel ilmiah yang terpublikasi melalui berbagai macam lembaga publikasi dengan pengelompokkan bidang tertentu. Pembaca artikel ilmiah perlu menemukan informasi dan gagasan penulis dari teks yang ditulis. Selain artikel ilmiah yang terpublikasi, terdapat juga artikel yang belum terpublikasi berupa paper, jurnal, atau artikel esai perlombaan mahasiswa.

#Dasar Teori

###Text Summarization

Text Summarization (Ringkasan Teks) merupakan teks yang terdiri atas satu atau dua kalimat yang menyampaikan informasi penting dari teks yang asli, biasanya jumlah kalimat tidak lebih dari setengah teks yang asli bahkan lebih sedikit

*   Extractive

mengidentifikasi bagian penting dari teks dan ditulis kembali dalam teks yang ringkas. Pendekatan ini akan bergantung pada kalimat-kalimat yang tercantum pada teks tersebut

*   Abstractive

mengidentifikasi dan mengintepretasikan bagian penting dari teks dengan membuat kalimat yang baru. Kalimat baru dari pendekatan ini dapat dimungkinkan tidak menggunakan kalimat yang tercantum tetapi yang lain dengan makna yang sama



#Repository File
Tools pembuatan program ini menggunakan python dengan menghubungan repository Github

In [ ]:
!git clone https://github.com/adhiutama17/MKBigData

In [ ]:
!ls MKBigData

In [ ]:
#File Path untuk Teks yang akan diolah
#Contoh file yang akan diolah telah ditempatkan di folder MKBigData/Teks/
pdf_path = 'MKBigData/Teks/Abdul Rohman_Universitas Airlangga_Provider R_D.pdf' #file dalam bentuk PDF maka diperlukan tools pdfminer

In [ ]:
pip install pdfminer.six

In [ ]:
from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfinterp import resolve1
from pdfminer.high_level import extract_text

file = open(pdf_path, 'rb') #membuka file pdf
parser = PDFParser(file)
document = PDFDocument(parser)

print(resolve1(document.catalog['Pages'])['Count']) #mendapatkan jumlah halaman, karena terdapat halaman yang tidak diperlukan nantinya

In [ ]:
def load_data(pdf_path, hal_awal, hal_akhir):
    """
    Membuat fungsi def untuk memanggil, membuka file pdf
    Input  : pdf_path (lokasi file), halaman awal (bukan cover) dan akhir (sebelum daftar pustaka)
    Output : kumpulan kalimat (disebut juga document)
    """
    documents_list = []
    file_pdf = open(pdf_path, 'rb') #membuka file pdf
    parser = PDFParser(file_pdf)
    document = PDFDocument(parser)
    file_text = extract_text(pdf_path, page_numbers = range(hal_awal,hal_akhir))
    text_split = file_text.split(". ")
    for sentence in text_split:
      text = line.strip()
      documents_list.append(text)
    return documents_list

    print('Jumlah Halaman: '+resolve1(document.catalog['Pages'])['Count']) #mendapatkan jumlah halaman, karena terdapat halaman yang tidak diperlukan nantinya